In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
#train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
# Batch and shuffle the data
#train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [10]:
def make_discriminator_model():
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation = "relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation = "softmax"))
    return model

In [11]:
discriminator_model = make_discriminator_model()

In [12]:
discriminator_model.compile(optimizer = "Adam" , loss = "categorical_crossentropy", metrics=["accuracy"])

In [13]:
discriminator_model.fit(train_images, tf.one_hot(train_labels, 10), epochs=5)

Epoch 1/5
1875/1875 [==============================] - 208s 110ms/step - loss: 0.7466 - accuracy: 0.8414
Epoch 2/5
1875/1875 [==============================] - 174s 93ms/step - loss: 0.1017 - accuracy: 0.9694
Epoch 3/5
1875/1875 [==============================] - 221s 118ms/step - loss: 0.0882 - accuracy: 0.9741
Epoch 4/5
1875/1875 [==============================] - 243s 129ms/step - loss: 0.0834 - accuracy: 0.9749
Epoch 5/5
1875/1875 [==============================] - 226s 121ms/step - loss: 0.0797 - accuracy: 0.9768


In [14]:
discriminator_model.save("CNN_V1_tfkerasmmodel")

INFO:tensorflow:Assets written to: CNN_V1_tfkerasmmodel/assets


# load model example

In [4]:
loaded_model = tf.keras.models.load_model("CNN_V1_tfkerasmmodel")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
import csv
import pandas as pd
import numpy as np
test = pd.read_csv('/Users/lishuyu/PycharmProjects/datasets/digit-recognizer/test.csv')

In [28]:
test.to_numpy().reshape(-1, 28, 28).shape
prediction = discriminator_model.predict(test.to_numpy().reshape(-1, 28, 28, 1))

In [44]:
# prediction = discriminator_model.predict(test_images)
# prediction = test_labels
labels = np.argmax(prediction, axis=1)
with open('submit.csv', 'w') as f:
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerow(["ImageId","Label"])
    for i in range(len(labels)):
        writer.writerow([i+1, labels[i]])
    # close the file
    f.close()


In [40]:
for i in range():
    print(labels[i])

2
0
9
0
3
7
0
3
0
3
5
7
4
0
4
3
3
1
9
0
9
1
1
5
7
4
2
7
4
7
7
5
4
2
6
2
5
5
1
6
7
7
4
9
8
7
8
2
6
7
6
8
8
3
8
2
1
2
2
0
4
1
7
0
0
0
1
9
0
1
6
5
8
8
2
8
9
9
2
3
5
4
1
8
9
2
4
3
6
7
2
0
6
6
1
4
3
9
7
4
0
9
2
0
7
3
0
5
0
8
0
0
4
7
1
7
1
1
3
3
3
7
2
8
6
3
8
7
8
4
3
5
6
0
0
0
3
1
3
6
4
3
4
5
5
8
7
7
2
8
4
3
5
6
5
3
7
5
7
8
3
0
4
5
1
3
7
6
3
0
2
7
8
6
1
3
7
4
1
2
4
8
5
2
4
9
2
1
6
0
6
1
4
9
6
0
9
7
6
9
1
9
0
9
9
0
8
4
6
2
0
9
3
6
3
2
1
6
3
4
2
3
1
2
2
0
4
6
1
0
0
4
9
1
7
3
2
3
8
6
8
6
2
8
5
5
4
8
3
5
9
7
1
3
8
4
5
1
4
5
6
3
3
5
7
0
6
8
3
1
6
0
6
3
9
5
1
5
8
4
0
9
2
0
5
3
7
1
9
9
5
7
7
9
9
6
3
0
3
3
6
9
8
2
6
3
7
1
4
5
8
5
9
0
0
3
8
4
1
8
4
1
1
9
8
4
5
1
5
3
6
3
1
3
0
9
0
0
6
0
6
3
1
8
6
0
6
5
2
2
6
7
7
2
5
8
3
9
2
4
8
6
3
8
4
2
3
8
1
6
4
8
7
9
7
6
9
5
3
7
6
5
5
4
2
6
2
1
3
7
1
7
9
9
6
1
1
1
7
3
9
7
6
1
1
1
9
3
8
5
5
0
4
1
2
3
1
1
3
5
9
6
6
5
3
1
4
7
4
7
4
8
5
2
6
1
3
9
5
0
8
4
7
4
4
4
1
5
3
9
5
7
6
9
5
9
2
3
5
6
6
7
5
0
5
1
7
4
4
1
1
4
9
5
6
0
1
3
1
0
4
8
1
2
7
9
4
8
3
7
7
4
2
4
6
7
6
3
2
0


3
4
1
1
4
3
3
0
0
3
2
4
1
5
9
2
4
9
5
8
9
9
9
1
8
0
0
9
0
8
8
0
9
8
6
2
3
2
0
6
3
8
1
9
2
0
4
2
8
9
4
3
6
0
2
0
4
5
3
4
1
2
2
4
9
4
3
2
1
9
1
6
1
0
3
4
0
1
9
4
0
0
4
8
1
9
2
3
3
4
3
7
0
7
9
7
4
5
9
7
7
0
4
2
2
7
8
5
0
4
9
8
8
2
4
1
4
1
4
1
0
9
9
5
6
4
9
1
2
6
0
3
0
2
3
0
4
0
1
0
8
4
9
3
7
1
4
1
7
6
0
7
7
1
5
0
3
6
0
2
8
8
8
5
7
7
2
9
6
7
8
8
6
7
3
1
7
3
1
8
0
1
0
4
3
7
2
9
7
9
3
6
2
3
4
1
3
0
9
9
2
6
4
0
0
7
5
1
8
3
8
1
6
5
8
9
3
2
0
7
1
6
7
1
6
8
4
3
5
9
1
3
5
3
1
9
2
5
8
6
3
6
0
9
2
5
1
3
7
4
4
4
2
0
6
3
1
0
4
3
6
1
7
6
2
7
1
7
0
6
9
6
5
1
3
6
8
1
9
8
6
6
9
3
6
8
2
4
0
6
1
8
5
1
4
0
9
9
9
4
8
4
1
6
4
9
1
8
9
2
3
4
6
1
0
1
6
7
9
0
0
8
1
4
2
5
2
8
4
6
9
7
1
8
3
5
4
9
3
3
5
3
0
0
5
9
2
5
3
7
6
5
9
6
5
0
2
1
3
8
5
7
7
0
7
7
1
5
6
7
0
4
3
0
0
0
0
7
6
8
8
6
7
1
3
1
0
2
3
5
1
8
5
1
7
6
3
6
6
8
3
8
5
1
5
2
6
5
1
5
8
3
5
5
3
2
3
8
5
8
6
7
0
3
8
8
4
2
6
4
5
8
3
0
5
1
1
3
0
9
6
7
8
2
5
3
6
2
0
6
5
1
2
9
7
0
2
5
5
6
1
4
1
3
1
5
5
0
1
9
3
9
5
0
7
9
8
3
0
1
3
5
8
1
8
5
7
0
9
6
4
8
3
0
6
0
4
5
8
2


0
6
3
6
6
6
5
1
9
9
3
6
3
2
0
7
3
0
3
2
5
7
9
6
4
8
4
4
1
9
8
8
9
1
5
7
1
6
7
0
2
8
2
4
5
5
2
6
1
1
4
0
9
4
3
7
9
3
0
2
9
6
3
3
3
2
3
4
6
7
9
3
6
7
8
3
9
1
2
4
5
9
9
4
1
4
7
6
8
2
8
0
6
2
9
6
7
1
2
7
8
6
0
8
5
1
7
7
3
5
3
7
4
7
0
4
1
9
1
5
2
1
6
6
0
9
6
6
0
2
1
7
9
5
8
2
7
2
3
0
2
1
9
9
9
1
3
3
6
9
3
3
7
4
1
8
1
7
5
4
2
8
0
6
4
6
4
7
4
6
7
8
0
2
3
3
8
7
2
3
4
4
9
6
8
6
6
1
2
8
1
7
6
7
1
8
7
6
2
9
7
5
8
4
6
8
0
1
7
3
2
0
8
8
3
5
4
1
8
6
4
1
0
1
1
0
2
4
5
6
4
3
8
6
2
6
9
8
4
6
5
6
2
5
7
4
9
1
3
8
2
8
4
9
5
8
8
4
2
6
1
5
5
3
6
7
0
3
4
6
6
5
6
1
3
7
7
2
7
3
3
3
9
1
2
1
5
1
1
6
0
2
4
6
9
6
6
5
1
0
0
7
6
9
5
1
7
6
9
7
4
8
2
3
6
8
6
3
2
1
0
8
9
2
4
4
0
0
9
6
8
0
3
9
3
9
7
9
7
3
7
8
8
2
3
8
4
1
2
7
2
1
2
7
1
7
0
5
3
5
5
3
3
8
6
9
9
0
2
5
7
4
6
8
6
2
3
3
7
8
8
4
9
7
0
3
8
7
4
8
2
8
7
7
5
7
8
0
7
5
1
5
9
9
0
4
9
7
4
7
9
0
5
4
3
5
9
0
9
4
4
8
6
1
2
1
1
9
2
7
7
0
1
5
1
6
9
8
3
4
0
3
1
0
5
3
9
8
1
7
8
5
6
8
1
1
7
1
4
8
0
8
5
6
0
7
7
1
6
8
0
6
2
1
0
5
4
5
1
2
1
1
2
7
3
3
6
5
2
8
5
9
8
4
3
5
3
4
0
6


9
0
0
0
8
2
4
3
2
8
2
6
7
4
6
4
3
2
0
9
8
6
4
3
0
3
0
3
0
2
9
2
1
8
4
7
7
2
7
3
1
3
7
0
3
7
2
0
2
0
7
9
1
2
0
5
6
2
3
6
2
4
9
4
5
8
9
3
7
4
2
9
4
8
6
7
0
3
8
7
1
3
1
6
5
0
6
0
1
9
6
4
0
8
8
7
4
0
9
9
6
8
0
2
4
8
4
4
1
7
1
1
9
2
2
7
6
8
0
2
0
6
0
6
9
0
3
7
7
7
9
5
0
3
8
9
8
0
1
4
6
7
9
7
7
9
7
6
2
8
2
1
4
9
7
5
8
1
2
1
1
7
1
9
3
0
1
9
9
6
0
1
8
5
0
7
4
6
6
2
9
2
1
3
5
1
9
3
8
9
3
4
1
1
8
9
9
7
5
4
1
6
0
8
2
6
0
9
0
6
8
7
2
0
3
4
7
8
4
6
0
0
5
8
2
3
3
5
1
2
1
3
8
7
2
8
2
1
0
4
4
4
2
3
6
1
4
4
1
1
6
5
7
1
5
4
1
5
9
4
3
3
3
4
0
7
4
1
3
8
1
1
6
5
7
3
4
5
4
5
6
4
3
5
7
5
9
3
8
0
8
9
6
8
5
7
9
0
9
7
2
9
1
3
8
1
4
7
3
8
1
5
7
5
0
2
7
3
1
0
1
3
2
2
2
4
0
0
4
9
5
9
1
7
0
1
4
5
8
3
0
4
0
9
6
9
0
2
0
7
3
5
6
8
2
9
1
6
1
5
1
9
1
9
5
8
9
9
6
3
8
7
9
5
3
9
3
7
1
8
7
9
3
3
9
9
0
2
3
9
4
7
6
4
3
5
1
9
7
4
4
1
5
1
0
5
9
9
1
6
5
6
4
8
3
3
3
7
9
7
5
1
2
2
0
9
9
1
5
5
7
9
4
2
1
2
8
4
0
2
8
3
2
8
8
3
4
4
1
8
1
2
0
4
4
6
9
6
1
9
0
3
5
5
5
2
8
5
8
1
9
8
9
4
4
0
8
8
2
3
4
2
7
2
0
7
6
8
9
9
0
5
9
2
3
1
9
2
9
2


1
5
4
5
8
9
9
1
1
7
9
1
7
7
6
5
8
2
7
1
3
6
8
5
2
7
1
4
0
8
5
8
5
1
3
4
5
5
2
1
3
5
2
2
9
4
4
9
5
9
1
3
2
9
6
2
2
0
7
5
7
3
2
9
4
6
2
9
2
2
8
1
0
0
9
0
6
2
4
1
0
4
3
1
1
4
9
0
8
2
9
5
0
2
3
0
7
2
2
2
0
1
1
1
7
6
1
7
7
8
0
5
0
9
1
0
2
3
4
9
9
2
6
5
0
5
3
7
3
3
4
4
7
8
1
9
3
4
5
2
7
4
3
5
7
4
1
8
4
3
7
6
2
3
0
7
1
4
3
9
9
1
7
5
6
8
4
1
1
5
2
2
2
8
4
0
7
6
4
3
2
3
1
7
6
7
6
8
9
0
7
3
6
9
6
9
4
5
9
5
9
7
1
5
9
3
2
7
6
7
8
1
6
1
8
9
7
7
1
6
0
5
8
5
7
1
4
7
2
1
7
6
6
2
4
2
3
5
5
2
9
0
2
8
3
7
8
2
0
7
2
7
2
9
1
2
0
2
9
3
0
6
2
8
0
3
7
4
1
4
0
3
1
4
5
7
9
3
1
9
5
0
4
7
2
2
1
9
5
7
2
6
2
6
7
7
8
0
5
2
7
0
2
0
2
0
2
4
2
5
9
2
3
2
7
5
1
4
2
5
6
1
9
8
8
9
4
5
5
8
8
6
8
6
5
0
0
6
9
5
4
4
2
2
2
2
1
7
7
8
0
0
1
6
5
4
2
0
2
5
7
8
5
2
4
4
9
4
4
6
6
4
0
3
9
1
3
2
4
0
0
1
1
7
6
2
5
1
2
7
5
1
8
0
1
5
7
8
5
9
1
2
7
7
0
0
1
7
3
0
3
1
4
1
3
8
3
7
3
9
1
7
7
1
2
6
5
8
0
3
2
0
6
5
4
6
7
0
3
4
6
3
5
2
7
7
5
5
4
9
0
3
1
5
3
1
2
0
2
4
8
9
4
2
9
2
5
9
5
3
4
2
6
6
5
2
9
9
7
1
7
7
4
4
8
8
7
6
1
4
4
9
0
9
6
7
2
9
4
1


8
5
9
8
2
9
4
3
2
9
2
9
4
1
7
2
1
3
3
7
1
1
7
7
9
9
3
5
8
3
5
1
4
6
9
4
0
3
6
3
6
6
2
9
9
9
7
8
6
6
0
5
1
3
1
4
6
2
0
2
2
2
8
3
2
0
1
0
1
7
6
9
4
9
9
4
7
8
9
7
9
6
3
9
5
0
8
7
0
6
8
8
5
7
5
4
7
8
8
3
3
2
3
6
3
8
5
5
2
5
2
4
2
9
3
9
9
2
6
9
6
0
0
9
0
8
2
1
7
6
2
0
1
6
2
3
8
9
1
2
5
8
2
0
7
7
1
1
0
8
6
7
4
4
7
3
2
0
9
4
2
7
7
1
3
4
1
3
6
3
6
6
9
5
2
4
5
1
3
6
4
4
2
0
2
7
1
9
8
2
4
6
6
3
6
8
2
8
6
1
8
7
3
6
6
5
7
0
2
1
3
4
9
4
3
9
2
7
1
3
2
0
3
3
9
0
4
0
6
9
5
3
7
4
5
0
1
5
0
4
2
3
4
0
1
0
5
8
4
0
3
5
8
6
9
4
4
7
4
3
5
0
9
2
3
6
9
8
2
5
8
7
0
2
2
5
6
7
2
2
5
1
8
5
7
2
2
8
3
7
7
5
6
4
4
1
5
8
9
0
8
8
2
3
6
2
9
8
0
2
0
5
4
8
7
4
6
6
2
5
2
2
6
2
4
0
7
6
2
1
2
6
4
4
5
2
6
9
9
6
2
4
9
8
2
4
9
6
7
8
1
0
9
8
6
4
3
8
4
7
5
1
9
1
4
3
0
4
8
0
9
0
0
3
2
7
1
4
2
8
4
7
8
4
3
1
0
3
2
4
2
2
7
9
4
1
4
2
3
9
8
2
9
1
6
6
9
6
1
4
0
9
9
3
9
6
8
4
0
7
0
0
2
8
3
1
1
3
7
0
6
7
7
0
0
6
0
3
9
5
4
6
1
7
6
3
9
8
7
8
0
4
3
8
0
1
7
7
3
0
6
2
8
2
8
4
9
6
3
4
3
9
7
1
3
2
6
2
5
5
2
9
6
8
6
7
0
8
9
4
0
2
1
5
1
6
8
1
0
2


In [39]:
labels = np.argmax(prediction, axis=1)

In [43]:
len(labels)

28000

In [45]:
len(test_images)

10000